In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(
            in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [4]:
conv_arch = ((1,64),(1,128), (2,256), (2,512),(2,512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(
            num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(7 * 7 * out_channels, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 10))

net = vgg(conv_arch)    

In [14]:
X = torch.rand(1, 1,224,224)
X.shape

torch.Size([1, 1, 224, 224])

In [15]:
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output size:\t', X.shape)

Sequential output size:	 torch.Size([1, 64, 112, 112])
Sequential output size:	 torch.Size([1, 128, 56, 56])
Sequential output size:	 torch.Size([1, 256, 28, 28])
Sequential output size:	 torch.Size([1, 512, 14, 14])
Sequential output size:	 torch.Size([1, 512, 7, 7])
Flatten output size:	 torch.Size([1, 25088])
Linear output size:	 torch.Size([1, 4096])
ReLU output size:	 torch.Size([1, 4096])
Dropout output size:	 torch.Size([1, 4096])
Linear output size:	 torch.Size([1, 4096])
ReLU output size:	 torch.Size([1, 4096])
Dropout output size:	 torch.Size([1, 4096])
Linear output size:	 torch.Size([1, 10])


In [16]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
batch_size = 64
lr = 0.05,
num_epochs = 10
trian_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())